tcmalloc: large alloc 1073750016 bytes == 0x58cc0000 @  0x7f7c81aa52a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


In [0]:
# we will verify that GPU is enabled for this notebook
# following should print: CUDA is available!  Training on GPU ...
# 
# if it prints otherwise, then you need to enable GPU: 
# from Menu > Runtime > Change Runtime Type > Hardware Accelerator > GPU

import torch
import numpy as np

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


##Download the data and unzip it

In [0]:
!wget -cq https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
!unzip -qq flower_data.zip

replace flower_data/valid/61/image_06296.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


PIL is the Python Imaging Library by Fredrik Lundh and Contributors.

In [0]:
!pip install Pillow==4.1.0
!pip install PIL
!pip install Image

  Using cached https://files.pythonhosted.org/packages/e7/14/ce7a5cc750ae3a8506d57997d4f18a0674fe45dd843882b9580f06b6d8f1/Pillow-4.1.0-cp36-cp36m-manylinux1_x86_64.whl
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.1.0 which is incompatible.
  Found existing installation: Pillow 5.4.0
    Uninstalling Pillow-5.4.0:
      Successfully uninstalled Pillow-5.4.0
  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL


##Import librarys

In [0]:
# Imports here
# google colab does not come with torch installed. And also, in course we are using torch 0.4. 
# so following snippet of code installs the relevant version

from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch
import matplotlib.pyplot as plt
import seaborn as sb

#%reload_ext autoreload        <------------— comment out 
#%autoreload 0                 <------------— comment out
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from PIL import Image
from torch.autograd import Variable
import json
from collections import OrderedDict

import math

plt.ion() 

##Data Path

In [0]:
data_dir ='./flower_data'
train_dir=data_dir+'/train'
valid_dir=data_dir+'/valid'

In [0]:
# TODO: Define your transforms for the training and validation sets

train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])
                                      ])
                
                                                             
valid_transforms = transforms.Compose([transforms.Resize(255),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406],
                                                          [0.229, 0.224, 0.225])
                                        ])
                                      
# TODO: Load the datasets with ImageFolder
image_datasets = torchvision.datasets.ImageFolder(train_dir, transform = train_transforms)
image_datasets2 = torchvision.datasets.ImageFolder(valid_dir, transform = valid_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
dataloaders = {
  'train' : torch.utils.data.DataLoader(image_datasets, batch_size=4, shuffle=True, num_workers=4),
  'valid': torch.utils.data.DataLoader(image_datasets2, batch_size=4, shuffle=True, num_workers=4)
}                              

In [0]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
    print(cat_to_name)

{'21': 'fire lily', '3': 'canterbury bells', '45': 'bolero deep blue', '1': 'pink primrose', '34': 'mexican aster', '27': 'prince of wales feathers', '7': 'moon orchid', '16': 'globe-flower', '25': 'grape hyacinth', '26': 'corn poppy', '79': 'toad lily', '39': 'siam tulip', '24': 'red ginger', '67': 'spring crocus', '35': 'alpine sea holly', '32': 'garden phlox', '10': 'globe thistle', '6': 'tiger lily', '93': 'ball moss', '33': 'love in the mist', '9': 'monkshood', '102': 'blackberry lily', '14': 'spear thistle', '19': 'balloon flower', '100': 'blanket flower', '13': 'king protea', '49': 'oxeye daisy', '15': 'yellow iris', '61': 'cautleya spicata', '31': 'carnation', '64': 'silverbush', '68': 'bearded iris', '63': 'black-eyed susan', '69': 'windflower', '62': 'japanese anemone', '20': 'giant white arum lily', '38': 'great masterwort', '4': 'sweet pea', '86': 'tree mallow', '101': 'trumpet creeper', '42': 'daffodil', '22': 'pincushion flower', '2': 'hard-leaved pocket orchid', '54': 's

In [0]:
# TODO: Build and train your network

#model = models.vgg19(pretrained=True)
model = models.resnet152(pretrained=True)
for param in model.parameters():
    param.requires_grad_(False)
    
#model = torchvision.models
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [0]:
from collections import OrderedDict
# TODO: Build and train your network

model.fc = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(2048, 1024)),
                          ('relu1', nn.ReLU()),
                          ('dropout1', nn.Dropout(p=0.3)),

    #                           ('fc2', nn.Linear(8192, 8192)),
#                           ('relu2', nn.ReLU()),
#                           ('dropout2', nn.Dropout(p=0.2)),
                          
                          ('fc2', nn.Linear(1024, 102)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
#model.classifier = classifier

In [0]:
from torch import nn, optim

model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)
#optimizer = optim.Adam(model.parameters(), lr=0.003)
model.to(device) # Transfer that model to the device we are using


cuda


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [0]:
epochs = 10
steps = 0
valid_loss_min = np.Inf

model.to(device)

# Training Loop
for epoch in range(epochs):
  
   
  running_loss = 0
  model.train()
  
  for images , labels in dataloaders['train']:
    steps += 1
    
    
   # print(device)
    images, labels = images.to(device), labels.to(device)
    
    optimizer.zero_grad()
    
    logps = model(images)
    loss = criterion(logps, labels)
    loss.backward()
    
    optimizer.step()
    
    running_loss += loss.item()
   # print(str(running_loss))
          
  print("After training, running loss = "+ str(running_loss))
  print("Starting Validation")
  #Validation Loop
  
  with torch.no_grad():
    model.eval()
    accuracy = 0 
    validation_loss = 0
    for images, labels in dataloaders['valid']:
      images, labels = images.to(device), labels.to(device)
      logps = model(images)

      loss = criterion(logps, labels)
      validation_loss+= loss.item()

      ps = torch.exp(logps)
      top_ps, top_class = ps.topk(1, dim=1)
      equals = top_class == labels.view(*top_class.shape)
      accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
     # print(top_class)
     # print(labels.view(*top_class.shape))
    
    #print(labels)
    print("Validation accuracy absolute: {:.3f}..".format(accuracy))
    print("Epoch {}/{}.. ".format(epoch+1, epochs),
            "Training loss: {:.3f}.. ".format(running_loss / len(dataloaders['train'])),  
            "Validation loss: {:.3f}..".format(validation_loss / len(dataloaders['valid'])),
            "Validation accuracy: {:.3f}..".format(accuracy / len(dataloaders['valid'])))

    model.train()
    
    
            
    if validation_loss <= valid_loss_min:
       print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
       valid_loss_min,
       validation_loss))
       torch.save(model.state_dict(), 'best_valid_loss_model.pt')
       valid_loss_min = validation_loss

After training, running loss = 7381.988890171051
Starting Validation
Validation accuracy absolute: 7.000..
Epoch 1/10..  Training loss: 4.507..  Validation loss: 4.497.. Validation accuracy: 0.034..
Validation loss decreased (inf --> 921.966657).  Saving model ...
After training, running loss = 7381.883078336716
Starting Validation
Validation accuracy absolute: 7.000..
Epoch 2/10..  Training loss: 4.507..  Validation loss: 4.499.. Validation accuracy: 0.034..
After training, running loss = 7381.9962339401245
Starting Validation
Validation accuracy absolute: 5.250..
Epoch 3/10..  Training loss: 4.507..  Validation loss: 4.499.. Validation accuracy: 0.026..
After training, running loss = 7381.660186290741
Starting Validation
Validation accuracy absolute: 5.250..
Epoch 4/10..  Training loss: 4.507..  Validation loss: 4.500.. Validation accuracy: 0.026..
After training, running loss = 7381.6271097660065
Starting Validation
Validation accuracy absolute: 5.250..
Epoch 5/10..  Training loss: 